In [101]:
import time
import konlpy
import openpyxl
import re
import pandas as pd
import requests
from konlpy.tag import Kkma
from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup



In [62]:
kma = Kkma()

In [63]:
wb = openpyxl.Workbook()
sheet = wb.active


In [ ]:
# 크롤링 대상

# 1. 제목
# 2. 본문 내용
# 3. 댓글 갯수 
# 4. 시간
# 5. 좋아요 갯수

# 에이전트+크롬디버깅모드 + 로그인 + 자게이동 + 다음 페이지 이동 코드
# @@@@절대 건들지마@@@@
# +개발자 도구 킨 상태로 실행 불가
# +자게 페이지 총 28408 개, 현재시점 5년까지만 존재

In [79]:

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


# id_input = (driver.find_element(By.XPATH,'//*[@id="container"]/form/p[1]/input')).click() #아이디


driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 
time.sleep(1)


driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동




/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


# 첫 번째 크롤링 방법 - 게시판 페이지 갯수 사용자 지정 = 실패

In [155]:
# while True:
#     print('page'+str(cnt))
#     if cnt > 300 : # 페이지 갯수 정하기 
#         break
    
#     cnt = cnt + 1

In [122]:
# every_url = 'http://everytime.kr/login'

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


# id_input = (driver.find_element(By.XPATH,'//*[@id="container"]/form/p[1]/input')).click() #아이디




driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 
time.sleep(1)


driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동



## 크롤링 함수 선언

title_data = []
etime_data = []
content_data = []
vote_data = []
comment_data = []

def crawling(pg):
    soup = BeautifulSoup(pg,"html.parser")
    to_body = soup.find(class_="wrap articles")
    title = to_body.find_all("h2")
    etime = to_body.find_all("time") ############## 시간 추가
    content = to_body.find_all("p")
    vote = to_body.find_all(class_="vote")
    comment = to_body.find_all(class_="comment")

    

    for i in range(0,len(title)):
        title_data.append(title[i].text)
        etime_data.append(etime[i].text) ############### 시간 추가
        content_data.append(content[i].text)
        vote_data.append(vote[i].text)
        comment_data.append(comment[i].text)

    return title_data, etime_data, content_data, vote_data, comment_data

# 6. 크롤링 시작

cnt = 0 # 첫번ㅈ째 페이지부터 순환하기 위함

title, etime, content, vote, comment = crawling(driver.page_source)
print("[INFO] page %d completed ... "%page_num)

while True:
    if cnt > 10 :
        break
    cnt = cnt + 1
    try:   
        driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 맞나?
        time.sleep(2)
        title, etime, content, vote, comment = crawling(driver.page_source)
        print("[INFO] page %d completed ... "%page_num)

    except:
        try:
            time.sleep(5)
            driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 맞나?
            time.sleep(1)
            title, etime, content, vote, comment = crawling(driver.page_source)
            printprint("[INFO] page %d completed ... "%page_num)
        except:
            print("[INFO] page %d is the last page. nice work!"%page_num)
            break
        
# 7 데이터프레임으로 변환 , 시각화

every_time_dict = {"title": title, "etime" : etime, "content": content, "vote": vote, "comment": comment}
every_time_dataframe = pd.DataFrame(every_time_dict)
display(every_time_dataframe)

# 8 csv format으로 저장

every_time_dataframe.to_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/data_only10.csv', index=False, header = True, encoding='utf-8-sig')





    




/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="userid"]"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000102dd6598 chromedriver + 4404632
1   chromedriver                        0x0000000102d5dfa3 chromedriver + 3911587
2   chromedriver                        0x0000000102a0ae6a chromedriver + 425578
3   chromedriver                        0x0000000102a42c91 chromedriver + 654481
4   chromedriver                        0x0000000102a42e91 chromedriver + 654993
5   chromedriver                        0x0000000102a7a274 chromedriver + 881268
6   chromedriver                        0x0000000102a60dcd chromedriver + 777677
7   chromedriver                        0x0000000102a77f79 chromedriver + 872313
8   chromedriver                        0x0000000102a60bd3 chromedriver + 777171
9   chromedriver                        0x0000000102a35ced chromedriver + 601325
10  c

In [117]:
df = pd.read_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/data_ansi.csv', encoding='cp949')
every_time_dataframe.to_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/data_noexcept.csv', index=False, header = True, encoding='cp949', sep='\t')

df.to_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/datatsv.csv',  )

# 두 번째 방법 : 모든 페이지 냅다하기 = 실패 , 1,2페이지만 반복되는 오류 발생

In [126]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 
time.sleep(1)


driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동

title_data = []
etime_data = []
content_data = []
vote_data = []
comment_data = []

def crawling(pg):
    soup = BeautifulSoup(pg,"html.parser")
    to_body = soup.find(class_="wrap articles")
    title = to_body.find_all("h2")
    etime = to_body.find_all("time") ############## 시간 추가
    content = to_body.find_all("p")
    vote = to_body.find_all(class_="vote")
    comment = to_body.find_all(class_="comment")

    

    for i in range(0,len(title)):
        title_data.append(title[i].text)
        etime_data.append(etime[i].text) ############### 시간 추가
        content_data.append(content[i].text)
        vote_data.append(vote[i].text)
        comment_data.append(comment[i].text)

    return title_data, etime_data, content_data, vote_data, comment_data


# 6. 크롤링 시작


page_num = 1 #이거 대신 아래 코드

title, etime, content, vote, comment = crawling(driver.page_source)
print("[INFO] page %d completed ... "%page_num)

---
while True:
    page_num += 1
    try:  
        driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 맞나?
        time.sleep(2)
        title, etime, content, vote, comment = crawling(driver.page_source)
        print("[INFO] page %d completed ... "%page_num)

    except:
        try:
            time.sleep(5)
            #기존 코드 driver.find_element_by_class_name("next").click()
            driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 맞나?
            time.sleep(1)
            title, etime, content, vote, comment = crawling(driver.page_source)
            printprint("[INFO] page %d completed ... "%page_num)
        except:
            print("[INFO] page %d is the last page. nice work!"%page_num)
            break
        
# 7 데이터프레임으로 변환 , 시각화

every_time_dict = {"title": title, "etime" : etime, "content": content, "vote": vote, "comment": comment}
every_time_dataframe = pd.DataFrame(every_time_dict)
display(every_time_dataframe)

# 8 csv format으로 저장

every_time_dataframe.to_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/data_noexcept.csv', index=False, header = True, encoding='cp949')


/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


[INFO] page 1 completed ... 
[INFO] page 2 completed ... 
[INFO] page 3 completed ... 
[INFO] page 4 completed ... 
[INFO] page 5 completed ... 
[INFO] page 6 completed ... 
[INFO] page 7 completed ... 
[INFO] page 8 completed ... 
[INFO] page 9 completed ... 
[INFO] page 10 completed ... 
[INFO] page 11 completed ... 
[INFO] page 12 completed ... 
[INFO] page 13 completed ... 
[INFO] page 14 completed ... 
[INFO] page 15 completed ... 
[INFO] page 16 completed ... 
[INFO] page 17 completed ... 
[INFO] page 18 completed ... 
[INFO] page 19 completed ... 
[INFO] page 20 completed ... 
[INFO] page 21 completed ... 
[INFO] page 22 completed ... 
[INFO] page 23 completed ... 
[INFO] page 24 completed ... 
[INFO] page 25 completed ... 
[INFO] page 26 completed ... 
[INFO] page 27 completed ... 
[INFO] page 28 is the last page. nice work!


,title,etime,content,vote,comment
0,방학때 알바하면 보통 얼마씩 벌어?,방금,순수 버는거만 쓰는거 말구,0,1
1,전자공 회로이론2,3분 전,나만 ㅈ된거 아니지?,0,1
2,게임하는 남자 거르는데,6분 전,그러면 한 80프로 걸러지나..?,0,7
3,나랑,7분 전,공장 들어갈 사람,0,1
4,소비자법과정책,8분 전,양 개 많은데 10문제정도밖에 안나오는거 실화냐..,0,0
...,...,...,...,...,...
535,다메다네~ 다메나노요~,21분 전,이 노래 알면 씹덕임?,0,5
536,밥 먹는데 마감 하는데 왜 안 나가냐는듯이,22분 전,청소 시끄럽게 하는 거 너무 기분 나빴다...ㅜㅜ 다시는 거기 안 갈래...,0,1
537,회계의이해 01반,23분 전,이번주 수업 비대면 아니지 ?? 그 중간 끝나고부터? 올대면이라고 하신 거죠 ??,0,0
538,나 불치병 걸린듯 ㅅㅂ...,23분 전,롤토체스 골드4에서 못빠져나오는중 골5 0 50 0 50 0 50 무한 로테이션중;...,0,2


# 세 번째 방법 = 성공

In [154]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 

# driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동

title_data = []
etime_data = []
content_data = []
vote_data = []
comment_data = []

def crawling(pg):
    soup = BeautifulSoup(pg,"html.parser")
    to_body = soup.find(class_="wrap articles")
    title = to_body.find_all("h2")
    etime = to_body.find_all("time") ############## 시간 추가
    content = to_body.find_all("p")
    vote = to_body.find_all(class_="vote")
    comment = to_body.find_all(class_="comment")


    for i in range(0,len(title)):
        title_data.append(title[i].text)
        etime_data.append(etime[i].text) ############### 시간 추가
        content_data.append(content[i].text)
        vote_data.append(vote[i].text)
        comment_data.append(comment[i].text)

    return title_data, etime_data, content_data, vote_data, comment_data

# 자게 이동 후 첫 페이지 크롤링

driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(5)

# driver.find_element(By.XPATH,'//*[@id="sheet"]/ul/li[3]/a').click() 아마도 없어진 기능인듯?

pg = driver.page_source

title, etime, content, vote, comment = crawling(pg)


print("1번째 페이지~")


# 두 번째 이후 크롤링

i = 1                                                                        

while True:
    if i > 30 :
        break
#     cnt = cnt + 1

    i += 1

    try:   
        driver.find_element(By.CLASS_NAME, "next").click()
        # 원래 코드 driver.find_element_by_class_name("next").click()

        time.sleep(1)

        pg = driver.page_source

        title, etime, content, vote, comment = crawling(pg)

        print("%d번째 페이지~"%(i))
        
        time.sleep(1)

    except:

        if driver.find_element(By.CLASS_NAME, "next") != None:            

            time.sleep(5)                           

            pg = driver.page_source         

            title, etime, content, vote, comment = crawling(pg)

            print("%d번째 페이지~(오류가 있었지만 해결!)"%(i))

        else:

            print("%d번째 페이지는 없습니다~ 끝!"%(i))

            break
            
# 7 데이터프레임으로 변환 , 시각화

every_time_dict = {"title": title, "etime" : etime, "content": content, "vote": vote, "comment": comment}

every_time_dataframe = pd.DataFrame(every_time_dict)
display(every_time_dataframe)

# 8 csv format으로 저장

every_time_dataframe.to_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/data.csv', sep = '\t', index=False, header = True)
            

/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


1번째 페이지~
2번째 페이지~
3번째 페이지~
4번째 페이지~
5번째 페이지~
6번째 페이지~
7번째 페이지~
8번째 페이지~
9번째 페이지~
10번째 페이지~
11번째 페이지~
12번째 페이지~
13번째 페이지~
14번째 페이지~
15번째 페이지~
16번째 페이지~
17번째 페이지~
18번째 페이지~
19번째 페이지~
20번째 페이지~
21번째 페이지~
22번째 페이지~
23번째 페이지~
24번째 페이지~
25번째 페이지~
26번째 페이지~
27번째 페이지~
28번째 페이지~
29번째 페이지~
30번째 페이지~
31번째 페이지~


,title,etime,content,vote,comment
0,낼 역학 쪽지시험 보는건가요??,방금,이번 주에 본다고 하셨던거 같은데 내일인지 수요일인지 모르겠어요,0,0
1,투자론 대박임,3분 전,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 하...,0,4
2,😡길거리에서 담배 피면 무적권 벌금,4분 전,"100,000원씩 물리면 안되나???간접흡연 너무 시러...ㅜㅜ 특히 아침에 집 나...",1,2
3,독문산,6분 전,듣는사람??,0,1
4,지금 사캠 과제 제출잘돼??,6분 전,나만 안돼?……🥹,0,0
...,...,...,...,...,...
595,셤기간에 볼링 딱 한판만 같이 칠 사람,10/22 18:07,아무나 구함,0,2
596,컴입 ㅎㅎㅇ 교수님 과제 도와주세요ㅠㅠ,10/22 18:06,과제7 하는데 너무 모르겠어요 도와주실 분..ㅜㅜ,0,1
597,일선 사이트,10/22 18:05,시험인데 안들어 가지는데 이거 어떡하냐 진짜 미치겠다ㅠㅠㅠㅠ,0,11
598,의류학과를 우습게 보고있네,10/22 18:04,용서 못해,0,2
